# 🎮 Exploração Interativa do Espaço Latente

**Tutorial de Espaço Latente - Notebook 5**

## 🎯 Objetivos
- Explorar interativamente o espaço latente
- Usar sliders para navegar pelas dimensões
- Gerar caminhadas (walks) no espaço latente
- Encontrar vizinhos mais próximos
- Interpolar entre amostras

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

from src.models.vae import VAE
from src.utils.data_loader import load_mnist
from src.utils.training import train_vae
from src.experiments.latent_explorer import LatentExplorer

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {DEVICE}")

## 📊 Preparação: Treinando um VAE

Vamos treinar rapidamente um VAE com dimensão latente maior para exploração.

In [ ]:
# Carrega dados
train_loader, val_loader, test_loader = load_mnist(batch_size=128)

# Cria e treina VAE
vae = VAE(input_dim=784, latent_dim=10, hidden_dims=[512, 256])

history = train_vae(
    model=vae,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=15,
    device=DEVICE,
    verbose=True
)

print("\n✅ Model trained!")

## 🎮 Criar o Explorer

In [ ]:
# Cria explorer
explorer = LatentExplorer(
    model=vae,
    latent_dim=10,
    device=DEVICE,
    latent_range=(-3, 3)
)

print(explorer)

## 🌊 Experimento 1: Grid Walk (Explorando 2 Dimensões)

Vamos explorar um grid 2D variando duas dimensões latentes.

In [ ]:
# Explora dimensões 0 e 1
grid = explorer.generate_grid_walk(
    dim1=0,
    dim2=1,
    n_steps=15,
    range_=(-3, 3)
)

print("\n💡 Observe como os dígitos mudam gradualmente ao longo do grid!")

In [ ]:
# Tenta outras combinações
explorer.generate_grid_walk(dim1=2, dim2=3, n_steps=15)
explorer.generate_grid_walk(dim1=4, dim2=5, n_steps=15)

## 🎨 Experimento 2: Interpolações Aleatórias

In [ ]:
# Gera interpolações entre amostras aleatórias
explorer.interpolate_between_samples(n_samples=5, steps=10)

print("\n💡 Cada linha mostra transição suave entre dois pontos aleatórios!")

## 🎲 Experimento 3: Amostras Aleatórias

In [ ]:
# Gera múltiplas amostras aleatórias
n_samples = 20
samples = []

for i in range(n_samples):
    z = explorer.sample_random()
    img = explorer.decode_latent(z)
    samples.append(img)

# Visualiza
fig, axes = plt.subplots(4, 5, figsize=(10, 8))
for i, ax in enumerate(axes.flat):
    ax.imshow(samples[i], cmap='gray')
    ax.axis('off')
plt.suptitle('Random Samples from Latent Space', fontweight='bold')
plt.tight_layout()
plt.show()

print("\n✅ Generated 20 random digits!")

## 🔍 Experimento 4: Vizinhos Mais Próximos

Dado um ponto aleatório no espaço latente, encontra imagens reais mais próximas.

In [ ]:
# Gera ponto aleatório
z_random = explorer.sample_random()

# Encontra vizinhos
nearest_imgs, nearest_labels, distances = explorer.find_nearest_sample(
    z_random,
    test_loader,
    n_nearest=5
)

print("\n💡 Mostra as 5 imagens reais mais próximas do ponto gerado!")

## 🎛️ Experimento 5: Exploração Interativa com Widgets

**Nota:** Funciona melhor em Jupyter Notebook com ipywidgets instalado.

In [ ]:
# Lança interface interativa (apenas em Jupyter com ipywidgets)
try:
    explorer.launch_interactive(backend='ipywidgets')
    print("\n🎮 Use os sliders para navegar pelo espaço latente!")
except Exception as e:
    print(f"⚠️ Interactive widgets não disponíveis: {e}")
    print("Instale com: pip install ipywidgets")
    print("E habilite: jupyter nbextension enable --py widgetsnbextension")

## 📊 Experimento 6: Análise Sistemática de Dimensões

Vamos variar cada dimensão individualmente para entender o que cada uma codifica.

In [ ]:
# Para cada dimensão latente, varia apenas ela
n_dims_to_analyze = min(6, vae.latent_dim)

fig, axes = plt.subplots(n_dims_to_analyze, 10, figsize=(15, 2*n_dims_to_analyze))

for dim in range(n_dims_to_analyze):
    values = np.linspace(-3, 3, 10)
    
    for i, val in enumerate(values):
        # Cria vetor latente com zeros, exceto dimensão atual
        z = torch.zeros(1, vae.latent_dim).to(DEVICE)
        z[0, dim] = val
        
        # Decodifica
        img = explorer.decode_latent(z)
        
        # Plota
        axes[dim, i].imshow(img, cmap='gray')
        axes[dim, i].axis('off')
        
        if i == 0:
            axes[dim, i].set_ylabel(f'Dim {dim}', rotation=0, labelpad=25,
                                    fontweight='bold')

plt.suptitle('Individual Latent Dimension Exploration', fontweight='bold', fontsize=14)
plt.tight_layout()
plt.show()

print("\n💡 Cada linha mostra como variar UMA dimensão afeta a geração!")

## 🧠 Experimento 7: Manipulação Aritmética no Espaço Latente

Operações tipo: digit_7 - digit_1 + digit_3 = ?

In [ ]:
# Pega exemplos de dígitos específicos
def get_latent_for_digit(digit, dataloader, n_samples=5):
    """Retorna códigos latentes de um dígito específico."""
    latents = []
    vae.eval()
    
    with torch.no_grad():
        for data, labels in dataloader:
            mask = labels == digit
            if mask.sum() > 0:
                data = data[mask].view(-1, 784).to(DEVICE)
                mu, _ = vae.encode(data)
                latents.append(mu.cpu())
                
                if len(torch.cat(latents)) >= n_samples:
                    break
    
    return torch.cat(latents)[:n_samples]

# Pega amostras
z_7 = get_latent_for_digit(7, test_loader, n_samples=3)
z_1 = get_latent_for_digit(1, test_loader, n_samples=3)
z_3 = get_latent_for_digit(3, test_loader, n_samples=3)

# Aritmética latente: 7 - 1 + 3
z_result = z_7.mean(dim=0, keepdim=True) - z_1.mean(dim=0, keepdim=True) + z_3.mean(dim=0, keepdim=True)

# Decodifica
result_img = explorer.decode_latent(z_result.to(DEVICE))

# Visualiza operação
fig, axes = plt.subplots(1, 5, figsize=(12, 3))

axes[0].imshow(explorer.decode_latent(z_7[0:1].to(DEVICE)), cmap='gray')
axes[0].set_title('7', fontweight='bold', fontsize=14)
axes[0].axis('off')

axes[1].text(0.5, 0.5, '-', ha='center', va='center', fontsize=30, fontweight='bold')
axes[1].axis('off')

axes[2].imshow(explorer.decode_latent(z_1[0:1].to(DEVICE)), cmap='gray')
axes[2].set_title('1', fontweight='bold', fontsize=14)
axes[2].axis('off')

axes[3].text(0.5, 0.5, '+', ha='center', va='center', fontsize=30, fontweight='bold')
axes[3].axis('off')

axes[4].imshow(result_img, cmap='gray')
axes[4].set_title('= ?', fontweight='bold', fontsize=14)
axes[4].axis('off')

plt.suptitle('Latent Space Arithmetic: 7 - 1 + 3', fontweight='bold', fontsize=16)
plt.tight_layout()
plt.show()

## 📝 Resumo

Neste notebook exploramos:

✅ Grid walks (variando 2 dimensões)  
✅ Interpolações aleatórias  
✅ Geração de amostras aleatórias  
✅ Busca de vizinhos mais próximos  
✅ Exploração interativa com widgets  
✅ Análise sistemática de dimensões  
✅ Aritmética no espaço latente  

**Aprendizados:**
- Espaço latente é contínuo e navegável
- Cada dimensão pode codificar características específicas
- Operações aritméticas funcionam (parcialmente) no espaço latente
- VAE permite exploração criativa

---

## 🚀 Próximo Notebook

No **Notebook 06**, veremos exemplos avançados e aplicações!

→→ `06_exemplos_avancados.ipynb`